In [1]:
import pandas as pd, numpy as np
from pathlib import Path
from sklearn.metrics import r2_score
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

base_path = Path()
big_data = base_path.joinpath('eda_data.csv')
big_df = pd.read_csv(big_data)

big_df['x10'] = big_df['x10'].str.strip('%').astype(float) / 100
big_df['x6'] = big_df['x6'].str.replace(r'[\)\$\,]','')
big_df['x6'] = big_df['x6'].str.replace(r'\(','-').astype(float)
big_df.dropna(inplace=True, how='any')
display(big_df.head(10))

C:\Users\sebas\AppData\Local\Temp\ipykernel_13036\3730334566.py:14: FutureWarning: The default value of regex will change from True to False in a future version.
  big_df['x6'] = big_df['x6'].str.replace(r'[\)\$\,]','')
C:\Users\sebas\AppData\Local\Temp\ipykernel_13036\3730334566.py:15: FutureWarning: The default value of regex will change from True to False in a future version.
  big_df['x6'] = big_df['x6'].str.replace(r'\(','-').astype(float)


,x0,x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,x11,x12,x13,y
0,-17.933519,6.559220,-14.452810,-4.732855,0.381673,2.563194,-1306.52,-89.394348,-28.454044,-16.201298,-0.0001,0.217010,9.729891,-0.786431,0.666146
1,-37.214754,10.774930,-15.384004,-0.077339,10.983774,-15.210206,-24.86,153.032652,-32.557736,69.675903,0.0000,-3.584908,35.727926,-0.985552,0.378411
2,0.330441,-19.609972,-9.167911,2.064124,12.071688,12.506141,-110.85,-141.437276,-20.794952,55.042604,0.0000,-3.991366,-9.283523,-3.394718,0.624498
3,-13.709765,-8.011390,6.759264,1.727615,-1.768382,24.039733,-324.43,51.039653,-7.046908,-31.424419,0.0001,7.908897,-2.891882,-2.690222,0.126622
4,-4.202598,7.076210,-26.004919,-4.269696,-3.414224,2.115989,1213.37,-31.046700,19.061182,-31.525515,-0.0001,0.846719,25.497480,3.516801,0.640025
5,12.570277,2.543883,25.673620,-0.842858,3.400531,37.176963,-1508.42,-98.956484,11.734073,100.654890,0.0000,15.359466,9.000782,1.929145,0.875990
6,17.949374,-35.406853,-25.164967,0.587439,-13.782894,-5.204895,-1431.44,-205.638274,16.399039,116.749751,0.0001,5.964466,15.851576,-0.819405,0.428533
7,38.645268,-21.871066,-74.405515,-9.707926,6.114417,-17.883435,1613.55,-239.151432,28.570116,-333.261028,0.0001,5.556269,-15.112181,2.480845,0.350310
8,35.279887,-2.004483,-12.071977,-1.020028,3.250741,-19.377852,1178.11,-108.229348,-29.886388,-126.434298,0.0000,4.085897,-10.558902,1.700618,0.612351
9,-31.756646,-5.499585,6.198889,-4.569550,13.902393,1.031500,-312.04,299.025699,24.338666,-25.001839,0.0000,4.204932,-5.320926,4.405564,0.494439


In [9]:
pca = PCA(n_components=14)
pca.fit(big_df.drop('y', axis=1))
display(pca.explained_variance_ratio_.round(3))
pca_df = pd.DataFrame(data=pca.transform(big_df.drop('y', axis=1)), columns=['PC1', 'PC2', 'PC3', 'PC4', 'PC5', 'PC6', 'PC7', 'PC8', 'PC9', 'PC10', 'PC11', 'PC12', 'PC13', 'PC14'])
pca_df.corr().round(4)

array([0.973, 0.016, 0.007, 0.001, 0.001, 0.001, 0.   , 0.   , 0.   ,
       0.   , 0.   , 0.   , 0.   , 0.   ])

,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,PC9,PC10,PC11,PC12,PC13,PC14
PC1,1.0,0.0,-0.0,0.0,-0.0,0.0,0.0,0.0,-0.0,-0.0,-0.0,-0.0,0.0,-0.0
PC2,0.0,1.0,-0.0,0.0,-0.0,0.0,0.0,0.0,-0.0,-0.0,-0.0,-0.0,0.0,0.0
PC3,-0.0,-0.0,1.0,-0.0,-0.0,-0.0,-0.0,-0.0,0.0,0.0,-0.0,0.0,0.0,-0.0
PC4,0.0,0.0,-0.0,1.0,-0.0,-0.0,-0.0,0.0,-0.0,-0.0,-0.0,0.0,-0.0,-0.0
PC5,-0.0,-0.0,-0.0,-0.0,1.0,0.0,0.0,0.0,-0.0,0.0,0.0,-0.0,0.0,0.0
PC6,0.0,0.0,-0.0,-0.0,0.0,1.0,0.0,0.0,-0.0,-0.0,-0.0,0.0,0.0,-0.0
PC7,0.0,0.0,-0.0,-0.0,0.0,0.0,1.0,0.0,-0.0,-0.0,-0.0,0.0,-0.0,0.0
PC8,0.0,0.0,-0.0,0.0,0.0,0.0,0.0,1.0,-0.0,-0.0,-0.0,-0.0,0.0,-0.0
PC9,-0.0,-0.0,0.0,-0.0,-0.0,-0.0,-0.0,-0.0,1.0,0.0,0.0,-0.0,-0.0,0.0
PC10,-0.0,-0.0,0.0,-0.0,0.0,-0.0,-0.0,-0.0,0.0,1.0,0.0,0.0,-0.0,-0.0


In [25]:
pca_df = pca_df[['PC1', 'PC2']].dropna()
scaler = StandardScaler()
scaler.fit(pca_df)
pca_df = pd.DataFrame(scaler.transform(pca_df), columns=['PC1', 'PC2'])

In [26]:
x = pca_df
y = big_df['y']
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.1, random_state=1)

In [27]:
model = LinearRegression()
model.fit(x_train, y_train)
y_pred = model.predict(x_test)
r2 = r2_score(y_test, y_pred)
print(f'R2 score: {r2}')

R2 score: 0.00041650267383985806


In [31]:
import torch, matplotlib.pyplot as plt
from torch.autograd import Variable

class linearRegression(torch.nn.Module):
    def __init__(self, inputSize, outputSize):
        super(linearRegression, self).__init__()
        self.linear = torch.nn.Linear(inputSize, outputSize)
        
    def forward(self, x):
        out = self.linear(x)
        return out
    
inputDim = 2
outputDim = 1
learningRate = 1e-3
epochs = 5_000

model = linearRegression(inputDim, outputDim)
if torch.cuda.is_available():
    model.cuda()

criterion = torch.nn.MSELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learningRate, momentum=0.9)

for epoch in range(epochs):
    if torch.cuda.is_available():
        x = Variable(torch.from_numpy(x_train.values).float().cuda())
        y = Variable(torch.from_numpy(y_train.values).float().cuda()).view(-1,1)
    else:
        x = Variable(torch.from_numpy(x_train.values).float())
        y = Variable(torch.from_numpy(y_train.values).float()).view(-1,1)
        
    optimizer.zero_grad()
    outputs = model(x)
    loss = criterion(outputs, y)
    loss.backward()
    optimizer.step()
    
    #print('epoch {}, loss {}'.format(epoch, loss.item()))

with torch.no_grad():
    if torch.cuda.is_available():
        predicted = model(Variable(torch.from_numpy(x_test.values).float().cuda())).cpu().data.numpy()
    else:
        predicted = model(Variable(torch.from_numpy(x_test.values).float())).data.numpy()
    
print(f'PyTorch R2 score: {r2_score(y_test, predicted)}')
    

PyTorch R2 score: 0.0004164669198430948
